In [22]:
from exactextract import exact_extract
import geopandas as gpd
import pandas as pd
import rasterio as rio
from tqdm import tqdm
import numpy as np

import os
import glob
import pathlib
from multiprocess import Pool
from typing import Union, List, Any
import itertools
import math

In [7]:

def _chunks(data: gpd.GeoDataFrame, n: int, raster: str, stats: List[str], include_cols: List[str]) -> List[Any]:
    """Yield successive n-sized chunks from a slice-able iterable."""
    for i in range(0, len(data), n):
        yield [data[i:i + n], raster, stats, include_cols]

def _zonal_stats_partial(feats: gpd.GeoDataFrame, raster: str, stats: List[str], include_cols: List[str]) -> List[Any]:
    from exactextract import exact_extract
    """Wrapper for zonal stats, takes a geodataframe"""
    return exact_extract(vec=feats, rast=raster, ops=stats, include_cols=include_cols, output="pandas")

def calculate_stats_parallel(polygon_layer_gdf: gpd.GeoDataFrame, raster: str, stats: List[str], pool: Pool, include_cols=['id'], n_jobs=16, index_column: str='id', prefix: str=''):
    """
    Calculate zonal statistics for a raster and a GeoDataFrame of polygons" in parallel.

    Args:
        polygon_layer_gdf (gpd.GeoDataFrame): A GeoDataFrame of polygons.
        raster (str): The path to the raster file.
        stats (List[str]): A list of statistics to calculate.
        pool (Pool): Pool of a jobs in multiprocess library.
        include_cols (List[str], optional): A list of columns to include in the output. Defaults to ['id'].
        n_jobs (int, optional): The number of cores to use for parallel processing. Defaults to 16.
        index_column (str, optional): The name of the index column. Defaults to 'id'.
        prefix (str, optional): A prefix string to add to the column names. Defaults to ''.

    Returns:
        pd.DataFrame: A DataFrame of zonal statistics.
    """
    # Define the number of cores to use for parallel processing
    
    # Use starmap and chunks for parallel processing
    stats_list = pool.starmap(_zonal_stats_partial,
                            _chunks(polygon_layer_gdf, round(len(polygon_layer_gdf) / pool._processes),
                                    raster,  stats, include_cols))
    stats_list = [df.set_index(index_column) for df in stats_list]
    calculated_stats = pd.concat(stats_list)
    
    if index_column is not None and index_column in include_cols:
        # change index dtype to dtype of index column in input layer
        index_dtype = str(polygon_layer_gdf[index_column].dtype)
        calculated_stats = calculated_stats.reset_index().astype({index_column:index_dtype})
        
    if len(prefix) > 0:
        # rename columns to include prefix string
        rename_dict = {stat: f"{prefix}{stat}" for stat in stats}
        calculated_stats = calculated_stats.rename(columns=rename_dict)
    
    return calculated_stats
    



In [3]:
n_jobs = 24
pool = Pool(n_jobs)
id_column = 'id'
stats = ['max', 'mean', 'variance', 'stdev']

# Paths to data
polygon_layer_path = r"../../../../Data/Wroclaw/Segmentacja/Segmenty_Wroclaw_2024_02_02.gpkg"
lidar_rasters_leafon_path = r'../../../../Data/Wroclaw/LIDAR/LeafOn/metryki_pnie'

In [4]:
# list rasters of lidar metrics
lidar_rasters_leafon_pathlib = pathlib.Path(lidar_rasters_leafon_path)
lidar_rasters_list = list(lidar_rasters_leafon_pathlib.rglob("*.tif"))

In [5]:
polygon_layer_gdf = gpd.read_file(polygon_layer_path, engine="pyogrio", use_arrow=True)
polygon_layer_gdf = polygon_layer_gdf.astype({'id':'int32'})

In [6]:
polygon_layer_stats_gdf = polygon_layer_gdf.copy(deep=True)

### Calculate metrics from a lidar rasters

In [8]:
for lidar_raster in tqdm(lidar_rasters_list):
    raster_name = lidar_raster.stem
    raster_rio = rio.open(lidar_raster)
    
    calculated_stats = calculate_stats_parallel(polygon_layer_gdf=polygon_layer_gdf, raster=lidar_raster,
                                            stats=stats, pool=pool, include_cols=[id_column], index_column=id_column, 
                                            n_jobs=n_jobs, prefix=f'{raster_name}_')
    polygon_layer_stats_gdf = pd.merge(polygon_layer_stats_gdf, calculated_stats, on='id', how='left')
    

100%|██████████| 175/175 [2:04:07<00:00, 42.56s/it] 


### Calculate other metrics

In [15]:
polygon_layer_stats_gdf['crownArea'] = polygon_layer_stats_gdf.geometry.area

In [19]:
# Define a function to calculate diameter
def calculate_diameter(area):
    return 2 * (math.sqrt(area / math.pi))

polygon_layer_stats_gdf['crown_diameter'] = polygon_layer_stats_gdf['crownArea'].apply(calculate_diameter)

In [25]:
polygon_layer_stats_gdf['crown_diameter_ln'] = np.log(polygon_layer_stats_gdf['crown_diameter'])
polygon_layer_stats_gdf['crownArea_ln'] = np.log(polygon_layer_stats_gdf['crownArea'])

In [9]:
polygon_layer_stats_gdf.columns

Index(['gid', 'id', 'treeID', 'crownArea', 'layer', 'path', 'geometry',
       'FHD_max', 'FHD_mean', 'FHD_variance',
       ...
       'zq95_variance', 'zq95_stdev', 'zsd_max', 'zsd_mean', 'zsd_variance',
       'zsd_stdev', 'zskew_max', 'zskew_mean', 'zskew_variance',
       'zskew_stdev'],
      dtype='object', length=707)

### Save layer

In [27]:
polygon_layer_stats_gdf.to_file(r"../../../../Data/Wroclaw/Segmentacja/Segmenty_statystyki_LeafOn.gpkg", engine="pyogrio")

In [26]:
polygon_layer_stats_gdf.to_parquet(r"../../../../Data/Wroclaw/Segmentacja/Segmenty_statystyki_LeafOn.parquet")